In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 13 mins
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.2
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,H2O_from_python_unknownUser_kdwqu7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.703 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [8]:
#file = '/content/drive/MyDrive/Colab Notebooks/Diplom/df4eda2clean2.csv'
file = 'df2prep4regressionV2.csv'
df = h2o.import_file(file)
df = df.drop('C1')
df = df.drop('MonthNo')
#df = df.drop('Year')
df = df.drop('Revenue_T')
df = df.drop('Zip')
df

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Units,Manufacturer,CategoryInSegmentInCompete
12,Natura,UrbanConvenienceNotCompete
7,Pirum,RuralProductivityNotCompete
3,VanArsdel,UrbanModerationIsCompete
3,Currus,RuralSelectNotCompete
3,Abbas,UrbanModerationIsCompete
2,Natura,YouthYouthNotCompete
2,Natura,YouthYouthNotCompete
2,Quibus,RuralSelectNotCompete
2,VanArsdel,UrbanModerationIsCompete
2,Currus,MixAll SeasonNotCompete


In [9]:
from h2o.estimators import H2OKMeansEstimator

# run h2o Kmeans to get good starting points
h2o_km = H2OKMeansEstimator(k=7, init="furthest", standardize=True)
h2o_km.train(training_frame=df)

user_points = h2o.H2OFrame(h2o_km.centers())

# show details
h2o_km.show()

kmeans Model Build progress: |███████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OKMeansEstimator : K-means
Model Key: KMeans_model_python_1695389858731_13


Model Summary: 
    number_of_rows    number_of_clusters    number_of_categorical_columns    number_of_iterations    within_cluster_sum_of_squares    total_sum_of_squares    between_cluster_sum_of_squares
--  ----------------  --------------------  -------------------------------  ----------------------  -------------------------------  ----------------------  --------------------------------
    877920            7                     2                                3                       1.2834e+06                       2.12178e+06             838380

ModelMetricsClustering: kmeans
** Reported on train data. **

Total Within Cluster Sum of Square Error: 1281725.5129442
Total Sum of Square Error to Grand Mean: 2121780.9909488484
Between Cluster Sum of Square Error: 840055.4780046484

Centroid Statistics: 
    centroid    size    within_cluster_sum_of_squares
--  ----------  ------  -------------------------------
    1           861624  1.21639e+06
    2           4       312.741
    3           47      2921.85
    4           185     4270.45
    5           20      1367.61
    6           586     7160.57
    7           15454   49299.3

Scoring History: 
    timestamp            duration    iterations    number_of_reassigned_observations    within_cluster_sum_of_squares
--  -------------------  ----------  ------------  -----------------------------------  -------------------------------
    2023-09-22 14:57:18  0.009 sec   0             nan                                  nan
    2023-09-22 14:57:20  1.898 sec   1             877920                               1.50961e+06
    2023-09-22 14:57:21  2.446 sec   2             12209                                1.35415e+06
    2023-09-22 14:57:21  2.575 sec   3             52                                   1.2834e+06

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [11]:
pred = h2o_km.predict(df)

kmeans prediction progress: |████████████████████████████████████████████████████| (done) 100%


In [12]:
df['cluster'] = pred
df

Units,Manufacturer,CategoryInSegmentInCompete,cluster
12,Natura,UrbanConvenienceNotCompete,3
7,Pirum,RuralProductivityNotCompete,5
3,VanArsdel,UrbanModerationIsCompete,6
3,Currus,RuralSelectNotCompete,6
3,Abbas,UrbanModerationIsCompete,6
2,Natura,YouthYouthNotCompete,6
2,Natura,YouthYouthNotCompete,6
2,Quibus,RuralSelectNotCompete,6
2,VanArsdel,UrbanModerationIsCompete,6
2,Currus,MixAll SeasonNotCompete,6


In [13]:
# To save as a parquet:
path = "df2clusteringV3.csv"
h2o.export_file(df, path)

Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
